<a href="https://colab.research.google.com/github/Chathura-Jayasinghe/EN3150_Assignment_03/blob/main/cnn_group_minions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>